## Final Project Submission

Please fill out: 
* Student name: Lynette Wangari
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Asha Deen
* Blog post URL:


# MICROSOFT MOVIE STUDIO EXPLORATION AND DATA ANALYSIS

### 1) Introduction

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies.  

Our task is to analyze the current movie genres at the Box office and their performance, considering various factors that affect movie success.

This analysis will  provide insights to help Microsoft make informed decisions on which types of films to focus on. To start, we'll gather and analyze relevant data and clarify business objectives. This will establish a solid foundation and direction for the project.

### 2) Problem Statement
Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. 

You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

### 3) Data Sets Used
The movie datasets are from the following websites:
1. https://www.boxofficemojo.com/
2. https://www.imdb.com/
3. https://www.rottentomatoes.com/
4. https://www.themoviedb.org/
5. https://www.the-numbers.com/


### 4) Project Structure

1. Data Understanding
2. Data Cleaning and Preparation
3. Exploratory Data Analysis
4. Visualization: Insights Derived from the Values
5. Recommendations: Key findings

Data Cleaning and Preparation
Exploratory Data Analysis
Visualization: Insights Derived from the Values
Recommendations: Key findings


#### 1. Data Understanding
 Our data is stored in a folder called zippedData. So we will first import the necessary packages then we'll explore the data by loading the datasets. 

In [74]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mticker
import seaborn as sns
%matplotlib inline

In [111]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('zippedData/im.db')

# Create a cursor object
cur = conn.cursor()

# Execute the SQL query to fetch the list of tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cur.fetchall()

# Print the list of tables
print(tables)

# Close the connection
conn.close()

[('movie_basics',), ('directors',), ('known_for',), ('movie_akas',), ('movie_ratings',), ('persons',), ('principals',), ('writers',)]


In [76]:
#import data
bom_movie_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
reviews_df = pd.read_csv('./zippedData/rt.reviews.tsv.gz',sep="\t", encoding = 'unicode_escape')
tmdb_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
budget_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')


In [77]:
import sqlite3
import pandas as pd

# Path to the SQLite database file
db_file_path = r'C:\Users\User\Documents\GitHub\dsc-phase-1-project-v2-4\zippedData\im.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_file_path)

# Read data from the movie_basics table
movie_basics = pd.read_sql("""
    SELECT *
    FROM movie_basics
""", conn)

# Display the first few rows of the DataFrame
movie_basics.head()



,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [78]:
movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [79]:
movie_ratings = pd.read_sql("""
    SELECT *
    FROM movie_ratings
""", conn)
movie_ratings.head(100)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
95,tt1515069,8.3,15
96,tt1515208,6.3,1447
97,tt1516577,5.3,147
98,tt1516595,4.4,64


In [80]:
movie_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


#### 2. Data Cleaning and Preparation


In [81]:
#cleaning data from Box Office Mojo
#fill missing values with zero
bom_movie_df['foreign_gross'] = bom_movie_df['foreign_gross'].fillna('0')

#remove commas from foreign gross and convert them to float
bom_movie_df['foreign_gross'] = bom_movie_df['foreign_gross'].str.replace(',', '').astype(float)

# Convert 'domestic_gross' column to float
bom_movie_df['domestic_gross'] = bom_movie_df['domestic_gross'].astype(float)

# Reorganize the columns
bom_movie_df = bom_movie_df[['title', 'year', 'domestic_gross', 'foreign_gross']]

In [82]:
bom_movie_df.head()

,title,year,domestic_gross,foreign_gross
0,Toy Story 3,2010,415000000.0,652000000.0
1,Alice in Wonderland (2010),2010,334200000.0,691300000.0
2,Harry Potter and the Deathly Hallows Part 1,2010,296000000.0,664300000.0
3,Inception,2010,292600000.0,535700000.0
4,Shrek Forever After,2010,238700000.0,513900000.0


In [83]:
# cleaning data from Movie Budgets
#  Convert 'release_date' column to datetime format
budget_df['release_date'] = pd.to_datetime(budget_df['release_date'])

# Convert 'production_budget', 'domestic_gross', and 'worldwide_gross' columns to integer format
def clean_currency(x):
   if isinstance(x, str):
        return int(x.replace('$','').replace(',',''))
   else:
        return x

budget_df['production_budget'] = budget_df['production_budget'].map(clean_currency)
budget_df['domestic_gross'] = budget_df['domestic_gross'].map(clean_currency)
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].map(clean_currency)

# Reorganize columns for better readability
budget_df = budget_df[['movie', 'release_date', 'production_budget', 'domestic_gross', 'worldwide_gross']]

In [84]:
budget_df.head()

,movie,release_date,production_budget,domestic_gross,worldwide_gross
0,Avatar,2009-12-18,425000000,760507625,2776345279
1,Pirates of the Caribbean: On Stranger Tides,2011-05-20,410600000,241063875,1045663875
2,Dark Phoenix,2019-06-07,350000000,42762350,149762350
3,Avengers: Age of Ultron,2015-05-01,330600000,459005868,1403013963
4,Star Wars Ep. VIII: The Last Jedi,2017-12-15,317000000,620181382,1316721747


In [85]:
#cleaning movie basics
movie_basics.isna().sum()

#drop the rows
movie_basics['runtime_minutes'].dropna()
movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [122]:
#clean the tmdb movies
#drop the original_title
if 'original_title' in tmdb_df.columns:
    tmdb_df.drop(['original_title'], axis=1, inplace=True)

#replace empty genre cells with category undecided
tmdb_df['genre_ids'].replace(to_replace ='[]', value='[7]', inplace=True)




C:\Users\User\AppData\Local\Temp\ipykernel_9420\810637660.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  tmdb_df['genre_ids'].replace(to_replace ='[]', value='[7]', inplace=True)


,Unnamed: 0,genre_ids,id,original_language,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,0.600,2018-06-22,Trailer Made,0.0,1


In [136]:
#dicitonary to translate genre numbers into actual genre names
genre_dict= {'7': 'undecided', '12': 'Adventure', '14': 'Fantasy', '16': 'Animation', '18': 'Drama', '27': 'Horror', '28': 'Action',
    '35': 'Comedy', '36': 'History', '37': 'Western', '53': 'Thriller', '80': 'Crime', '99': 'Documentary', 
    '878': 'Sci-Fi', '9648': 'Mystery', '10402': 'Music', '10749': 'Romance', '10751': 'Family', '10752': 'War', '10770': 'TV'

}
tmdb_df
# Function to convert genre numbers into genre names
def convert_genre(genre_ids):
    description = ''
    for num in genre_ids.strip('][').split(', '):
        if num in genre_dict:
            description += genre_dict[num] + ','
    return description.rstrip(',')

# Convert genre numbers to genre names
tmdb_df['genre_ids'] = tmdb_df['genre_ids'].map(convert_genre)

# Convert 'release_date' column to datetime
tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'])

# Drop duplicate entries
tmdb_df.drop_duplicates(inplace=True)
tmdb_df

C:\Users\User\AppData\Local\Temp\ipykernel_9420\1901863793.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmdb_df['genre_ids'] = tmdb_df['genre_ids'].map(convert_genre)
C:\Users\User\AppData\Local\Temp\ipykernel_9420\1901863793.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmdb_df['release_date'] = pd.to_datetime(tmdb_df['release_date'])
C:\Users\User\AppData\Local\Temp\ipykernel_9420\1901863793.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

,title,release_date,genre_ids,popularity,vote_average,vote_count
0,Harry Potter and the Deathly Hallows: Part 1,2010-11-19,,33.533,7.7,10788
1,How to Train Your Dragon,2010-03-26,,28.734,7.7,7610
2,Iron Man 2,2010-05-07,,28.515,6.8,12368
3,Toy Story,1995-11-22,,28.005,7.9,10174
4,Inception,2010-07-16,,27.920,8.3,22186
...,...,...,...,...,...,...
26512,Laboratory Conditions,2018-10-13,,0.600,0.0,1
26513,_EXHIBIT_84xxx_,2018-05-01,,0.600,0.0,1
26514,The Last One,2018-10-01,,0.600,0.0,1
26515,Trailer Made,2018-06-22,,0.600,0.0,1


Merge Data sets to get information on movie, title release date, runtime, votes, actor and director

In [86]:
#join the movie basics table and movie ratings
movie_basics_ratings_merge = pd.read_sql('''
    SELECT 
        movie_id,
        primary_title AS title,
        start_year AS year,
        runtime_minutes AS runtime,
        genres,
        averagerating AS avg_rating,
        numvotes
    FROM 
        movie_basics
    LEFT JOIN 
        movie_ratings
    USING(movie_id)
''', conn)
movie_basics_ratings_merge.head()

,movie_id,title,year,runtime,genres,avg_rating,numvotes
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [87]:
#merge the principals and persons tables
directors = pd.read_sql('''
    SELECT 
        principals.movie_id, 
        persons.primary_name AS director
    FROM 
        principals
    JOIN 
        persons
    USING(person_id)
    WHERE 
        category = 'director'
''', conn)

directors.head()

,movie_id,director
0,tt0111414,Frank Howson
1,tt0323808,Robin Hardy
2,tt0417610,Alejandro Chomski
3,tt0469152,Alyssa R. Bennett
4,tt0473032,J. Neil Schulman


In [88]:
#group the directors
directors = directors.drop_duplicates()

#define a separator for joining 
separator = ','

#group the directors by movie id and concatenate their names 
directors = directors.groupby('movie_id').agg(lambda x: separator.join(x.tolist())).reset_index()

In [89]:
directors.head(20)

,movie_id,director
0,tt0063540,Harnam Singh Rawail
1,tt0066787,Mani Kaul
2,tt0069049,Orson Welles
3,tt0069204,Hrishikesh Mukherjee
4,tt0100275,"Raoul Ruiz,Valeria Sarmiento"
5,tt0111414,Frank Howson
6,tt0112502,Mc Jones
7,tt0139613,"António Loja Neves,José Manuel Alves Pereira"
8,tt0146592,Ágnes Kocsis
9,tt0154039,Miklós Jancsó


In [90]:
#get actors and from the principals and actors table
# Select actors from principals table where the category is 'actor' and order by movie_id
actors_query = '''
SELECT movie_id, primary_name AS actors
FROM principals
JOIN persons USING(person_id)
WHERE category = 'actor'
ORDER BY movie_id
'''

# Read data from SQL query into a DataFrame
actors = pd.read_sql(actors_query, conn)

# Remove duplicate entries
actors = actors.drop_duplicates()

# Group actors by movie_id and concatenate their names into a single string separated by ','
actors_grouped = actors.groupby('movie_id')['actors'].apply(lambda x: ', '.join(x)).reset_index()


In [91]:
actors.head(20)

,movie_id,actors
0,tt0063540,Dilip Kumar
1,tt0063540,Balraj Sahni
2,tt0063540,Sanjeev Kumar
3,tt0066787,Arun Khopkar
4,tt0066787,Om Shivpuri
5,tt0069049,John Huston
6,tt0069049,Peter Bogdanovich
7,tt0069204,Rabi Ghosh
8,tt0069204,Vijay Arora
9,tt0069204,Asrani


In [92]:
#merge the dataframe containing movie information with that containing directord
merged_df_directors = pd.merge(movie_basics_ratings_merge ,directors, how = 'left', on = 'movie_id')
    


In [93]:
merged_df_directors.head()

,movie_id,title,year,runtime,genres,avg_rating,numvotes,director
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,Orson Welles
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,Hrishikesh Mukherjee
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"Raoul Ruiz,Valeria Sarmiento"


In [94]:
sql_df = pd.merge(merged_df_directors, actors, how = 'left', on = 'movie_id')

In [95]:
sql_df.head()

,movie_id,title,year,runtime,genres,avg_rating,numvotes,director,actors
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Dilip Kumar
1,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Balraj Sahni
2,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Sanjeev Kumar
3,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Arun Khopkar
4,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Om Shivpuri


In [96]:

sql_df.dropna(subset=['genres'], inplace = True)
#remove duplicates
sql_df.drop_duplicates(inplace=True)

In [97]:
sql_df.head()

,movie_id,title,year,runtime,genres,avg_rating,numvotes,director,actors
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Dilip Kumar
1,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Balraj Sahni
2,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Harnam Singh Rawail,Sanjeev Kumar
3,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Arun Khopkar
4,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama",7.2,43.0,Mani Kaul,Om Shivpuri


##### Merger Numbers and Box Office to source revenue for every movie

In [112]:


# Merge 'Box Office Mojo' with 'The Numbers' on a common column
bom_budget = pd.merge(bom_movie_df, budget_df, how='outer', left_on='title', right_on='movie')



In [99]:
bom_budget.head(100)

,title,year,domestic_gross_x,foreign_gross,movie,release_date,production_budget,domestic_gross_y,worldwide_gross
0,NaN,NaN,NaN,NaN,#Horror,2015-11-20,1500000.0,0.0,0.0
1,'71,2015.0,1300000.0,355000.0,NaN,NaT,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,(500) Days of Summer,2009-07-17,7500000.0,32425665.0,34439060.0
3,"1,000 Times Good Night",2014.0,53900.0,0.0,NaN,NaT,NaN,NaN,NaN
4,10 Cloverfield Lane,2016.0,72100000.0,38100000.0,10 Cloverfield Lane,2016-03-11,5000000.0,72082999.0,108286422.0
...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,"4 luni, 3 saptamani si 2 zile",2008-01-25,900000.0,1196321.0,11993156.0
96,NaN,NaN,NaN,NaN,4/11,2015-01-01,250000.0,0.0,0.0
97,NaN,NaN,NaN,NaN,40 Days and 40 Nights,2002-03-01,17000000.0,37939782.0,95092667.0
98,NaN,NaN,NaN,NaN,40 Weeks,2014-12-31,900000.0,0.0,0.0


In [100]:
#Fill missing values in the 'title' column with values from the 'movie' column
bom_budget['title'] = bom_budget['title'].fillna(bom_budget['movie'])

In [101]:
#fill missing values in the domestic_gross column
bom_budget['domestic_gross_x'].fillna(bom_budget['domestic_gross_y'], inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_9420\746476591.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bom_budget['domestic_gross_x'].fillna(bom_budget['domestic_gross_y'], inplace=True)


In [102]:
bom_budget.head()

,title,year,domestic_gross_x,foreign_gross,movie,release_date,production_budget,domestic_gross_y,worldwide_gross
0,#Horror,NaN,0.0,NaN,#Horror,2015-11-20,1500000.0,0.0,0.0
1,'71,2015.0,1300000.0,355000.0,NaN,NaT,NaN,NaN,NaN
2,(500) Days of Summer,NaN,32425665.0,NaN,(500) Days of Summer,2009-07-17,7500000.0,32425665.0,34439060.0
3,"1,000 Times Good Night",2014.0,53900.0,0.0,NaN,NaT,NaN,NaN,NaN
4,10 Cloverfield Lane,2016.0,72100000.0,38100000.0,10 Cloverfield Lane,2016-03-11,5000000.0,72082999.0,108286422.0


In [104]:
#calculate the foreign by subtracting domestic_groos_y from worldwide_gross
bom_budget['foreign'] = bom_budget['worldwide_gross'] - bom_budget['foreign_gross']
bom_budget['foreign'].fillna(bom_budget['foreign'], inplace=True)


In [105]:
#fill missing values in the release_date column with converted datetime
bom_budget['release_date'].fillna(pd.to_datetime(bom_budget['year'], format='%Y'), inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_9420\1853197591.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bom_budget['release_date'].fillna(pd.to_datetime(bom_budget['year'], format='%Y'), inplace=True)


In [ ]:
bom_budget.head()

,title,release_date,domestic_gross,foreign,worldwide_gross
1,'71,2015-01-01,1300000.0,355000.0,1655000.0
3,"1,000 Times Good Night",2014-01-01,53900.0,0.0,53900.0
4,10 Cloverfield Lane,2016-03-11,72100000.0,70186422.0,108286422.0
7,10 Years,2012-01-01,203000.0,0.0,203000.0
9,1001 Grams,2015-01-01,11000.0,0.0,11000.0


In [106]:
#fill missing values 
bom_budget['worldwide_gross'].fillna(bom_budget['domestic_gross_x'] + bom_budget['foreign'], inplace=True)
#reorganize columns and drop unnecessary columns
bom_budget = bom_budget[['title', 'release_date', 'domestic_gross_x','foreign', 'worldwide_gross']]
bom_budget.rename(columns={'domestic_gross_x': 'domestic_gross'}, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_9420\256490549.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bom_budget['worldwide_gross'].fillna(bom_budget['domestic_gross_x'] + bom_budget['foreign'], inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_9420\256490549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bom_budget.rename(

In [107]:
#drop al rows with missing values 
bom_budget.dropna(inplace=True)
bom_budget = bom_budget[bom_budget['worldwide_gross'] !=0.0]

C:\Users\User\AppData\Local\Temp\ipykernel_9420\4195427511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bom_budget.dropna(inplace=True)


In [108]:
bom_budget

,title,release_date,domestic_gross,foreign,worldwide_gross
4,10 Cloverfield Lane,2016-03-11,72100000.0,70186422.0,1.082864e+08
18,12 Strong,2018-01-19,45800000.0,49518378.0,7.111838e+07
19,12 Years a Slave,2013-10-18,56700000.0,49925343.0,1.810253e+08
20,127 Hours,2010-11-05,18300000.0,17817171.0,6.021717e+07
23,13 Hours: The Secret Soldiers of Benghazi,2016-01-15,52900000.0,52811370.0,6.941137e+07
...,...,...,...,...,...
7914,Zookeeper,2011-07-08,80400000.0,81305525.0,1.708055e+08
7916,Zoolander 2,2016-02-12,28800000.0,27448693.0,5.534869e+07
7918,Zootopia,2016-03-04,341300000.0,336929616.0,1.019430e+09
7922,[Rec] 2,2010-07-09,27800.0,27766.0,1.852777e+07


#### Merge the two dataframes

In [109]:
#merge the two dataframes
